In [10]:
import re, string, unicodedata
import numpy as np
import pandas as pd
from pprint import pprint
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
stop_words = stopwords.words('english')

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
from gensim.corpora import Dictionary


nltk.download('wordnet')
from nltk.stem.porter import PorterStemmer
from nltk.stem.wordnet import WordNetLemmatizer
#from bs4 import BeautifulSoup
#from nltk import word_tokenize, sent_tokenize

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
# %matplotlib inlin

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Kornelius\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


# Import data and prepare data

In [11]:
# Load dataset
df = pd.read_csv('C:/Users/Kornelius/Desktop/Data 2/nips-papers/papers.csv', header = 0, sep = ';', error_bad_lines=False)
# Drop the columns not needed
df = df.drop(columns=['id', 'event_type', 'pdf_name'], axis=1)

In [12]:
# Drop a row by condition
df = df[df.abstract != 'Abstract Missing']
# Print out the first rows of papers

In [13]:
data = df.abstract.values.tolist()

In [14]:
from nltk.corpus import stopwords 
from nltk.stem.wordnet import WordNetLemmatizer
import string
stop = set(stopwords.words('english'))
exclude = set(string.punctuation) 
ps = PorterStemmer()
def clean(data):
    stop_free = " ".join([i for i in data.lower().split() if i not in stop])
    punc_free = ''.join(ch for ch in stop_free if ch not in exclude)
    normalized = " ".join(ps.stem(word) for word in punc_free.split())
    return normalized

data = [clean(data).split() for data in data]  

In [15]:
# remove characters and numbers
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

In [16]:
data = data_words
# Create Dictionary
id2word = corpora.Dictionary(data)
print((id2word))

Dictionary(12827 unique tokens: ['activ', 'also', 'analysi', 'appli', 'assum']...)


In [17]:
# Create Corpus
texts = data
# Term Document Frequency and creating corpus
corpus = [id2word.doc2bow(text) for text in texts]

Gensim creates a unique id for each word in the document. The produced corpus shown above is a mapping of (word_id, word_frequency).
For example, (0, 1) above implies, word id 0 occurs once in the first document. Likewise, word id 1 occurs twice and so on.
This is used as the input by the LDA model.

We have everything required to train the LDA model. In addition to the corpus and dictionary, you need to provide the number of topics as well.

Apart from that, alpha and eta are hyperparameters that affect sparsity of the topics. According to the Gensim docs, both defaults to 1.0/num_topics prior.

chunksize is the number of documents to be used in each training chunk. update_every determines how often the model parameters should be updated and passes is the total number of training passes.

In [18]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=1, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [19]:
lda_model5 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=5, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model10 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model20 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model50 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=50, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

lda_model100 = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=100, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


C:\Users\Kornelius\Anaconda3\lib\site-packages\gensim\models\ldamodel.py:775: RuntimeWarning: divide by zero encountered in log
  diff = np.log(self.expElogbeta)


In [20]:
# Print the Keyword in the 1 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.014*"model" + 0.013*"algorithm" + 0.013*"learn" + 0.010*"use" + '
  '0.010*"problem" + 0.010*"method" + 0.008*"data" + 0.008*"show" + '
  '0.007*"propos" + 0.007*"gener"')]


In [21]:
# Print the Keyword in the 5 topics
pprint(lda_model5.print_topics())
doc_lda = lda_model5[corpus]

# Print the Keyword in the 10 topics
pprint(lda_model10.print_topics())
doc_lda = lda_model10[corpus]

# Print the Keyword in the 20 topics
pprint(lda_model20.print_topics())
doc_lda = lda_model20[corpus]

# Print the Keyword in the 50 topics
pprint(lda_model50.print_topics())
doc_lda = lda_model50[corpus]

# Print the Keyword in the 100 topics
pprint(lda_model100.print_topics())
doc_lda = lda_model100[corpus]

[(0,
  '0.029*"bound" + 0.025*"algorithm" + 0.016*"regret" + 0.015*"learn" + '
  '0.015*"problem" + 0.013*"set" + 0.011*"optim" + 0.010*"onlin" + '
  '0.010*"studi" + 0.010*"polici"'),
 (1,
  '0.018*"algorithm" + 0.016*"method" + 0.014*"problem" + 0.012*"learn" + '
  '0.012*"use" + 0.011*"propos" + 0.010*"show" + 0.010*"estim" + 0.010*"data" '
  '+ 0.010*"model"'),
 (2,
  '0.042*"model" + 0.014*"system" + 0.012*"data" + 0.011*"activ" + 0.011*"use" '
  '+ 0.010*"dynam" + 0.010*"network" + 0.009*"process" + 0.009*"observ" + '
  '0.009*"signal"'),
 (3,
  '0.034*"graph" + 0.034*"matrix" + 0.027*"cluster" + 0.019*"rank" + '
  '0.009*"algorithm" + 0.009*"complet" + 0.008*"recov" + 0.008*"node" + '
  '0.008*"norm" + 0.008*"matric"'),
 (4,
  '0.027*"learn" + 0.023*"imag" + 0.021*"model" + 0.019*"train" + '
  '0.019*"label" + 0.018*"network" + 0.016*"task" + 0.012*"use" + '
  '0.011*"object" + 0.010*"featur"')]
[(0,
  '0.030*"submodular" + 0.021*"rule" + 0.020*"search" + 0.018*"belief" + '
  '0

# Evaluation
Topic 0 is a represented as _0.016“car” + 0.014“power” + 0.010“light” + 0.009“drive” + 0.007“mount” + 0.007“controller” + 0.007“cool” + 0.007“engine” + 0.007“back” + ‘0.006“turn”.

It means the top 10 keywords that contribute to this topic are: ‘car’, ‘power’, ‘light’.. and so on and the weight of ‘car’ on topic 0 is 0.016.

The weights reflect how important a keyword is to that topic.

Looking at these keywords, can you guess what this topic could be? You may summarise it either are ‘cars’ or ‘automobiles’.

In [22]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity5: ', lda_model5.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity10: ', lda_model10.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity20: ', lda_model20.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity50: ', lda_model50.log_perplexity(corpus))  # a measure of how good the model is. lower the better.
print('\nPerplexity100: ', lda_model100.log_perplexity(corpus))  # a measure of how good the model is. lower the better.


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100: ', coherence_lda)


# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence ScoreUMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model5, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score5UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model10, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score10UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model20, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score20UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model50, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score50UMass: ', coherence_lda)

coherence_model_lda = CoherenceModel(model=lda_model100, texts=data, dictionary=id2word, coherence='u_mass')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score100UMass: ', coherence_lda)


Perplexity:  -7.3370528837678055

Perplexity5:  -7.311895204650633

Perplexity10:  -7.372718990378462

Perplexity20:  -7.3834574983049714

Perplexity50:  -7.420792239026178

Perplexity100:  -21.60733735376125

Coherence Score:  0.263846120041168

Coherence Score5:  0.40416796057366317

Coherence Score10:  0.41489065613483633

Coherence Score20:  0.3985465790172792

Coherence Score50:  0.4112285725887475

Coherence Score100:  0.4135019669962473

Coherence ScoreUMass:  -1.1975019153673416

Coherence Score5UMass:  -2.035671328726481

Coherence Score10UMass:  -5.282097582661903

Coherence Score20UMass:  -6.46925754585322

Coherence Score50UMass:  -9.395029709372729

Coherence Score100UMass:  -12.071492459401174
